In [1]:
import sys 
assert sys.version_info >= (3,7)

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from util_func import *


## Color Segmentation for Square traffic sign

In [ ]:
#use this(this is the latest)

In [2]:
import cv2 as cv
import numpy as np

# Read the image
img = cv.imread('image_project2/031_0001.png')

# Convert image from BGR to the HSV color space
hsv_image = cv.cvtColor(img, cv.COLOR_BGR2HSV)

# Define upper and lower bounds for the light blue color in HSV
lower_blue = np.array([90, 100, 100])
upper_blue = np.array([110, 255, 255])

# Define upper and lower bounds for the red color in HSV
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([180, 255, 255])

# Define upper and lower bounds for the yellow color in HSV
lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([30, 255, 255])

# Create binary masks for each color
mask_blue = cv.inRange(hsv_image, lower_blue, upper_blue)
mask_red1 = cv.inRange(hsv_image, lower_red1, upper_red1)
mask_red2 = cv.inRange(hsv_image, lower_red2, upper_red2)
mask_yellow = cv.inRange(hsv_image, lower_yellow, upper_yellow)

# Combine the masks for different colors
combined_mask = mask_blue | mask_red1 | mask_red2 | mask_yellow

# Morphological operations to clean up the mask
kernel = np.ones((5, 5), np.uint8)
combined_mask = cv.morphologyEx(combined_mask, cv.MORPH_OPEN, kernel)
combined_mask = cv.morphologyEx(combined_mask, cv.MORPH_CLOSE, kernel)

# Find contours in the binary mask
contours, _ = cv.findContours(combined_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# Filter contours based on area to find the largest contour (traffic sign)
largest_contour = max(contours, key=cv.contourArea)

# Create a mask for the traffic sign region
sign_mask = np.zeros_like(img)
cv.drawContours(sign_mask, [largest_contour], -1, (255, 255, 255), thickness=cv.FILLED)

# Extract the segmented traffic sign from the original image using the mask
seg_traffic_sign = cv.bitwise_and(img, sign_mask)

# Display the segmented traffic sign
cv.imshow('Segmented Traffic Sign', seg_traffic_sign)
cv.waitKey(0)
cv.destroyAllWindows()


## Color segmentation + edge detection 

In [3]:

# Read the image
img = cv.imread('image_project2/031_0001.png')

# Convert image from BGR to the HSV color space
hsv_image = cv.cvtColor(img, cv.COLOR_BGR2HSV)

# Define upper and lower bounds for the light blue color in HSV
lower_blue = np.array([90, 100, 100])
upper_blue = np.array([110, 255, 255])

# Define upper and lower bounds for the red color in HSV
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([180, 255, 255])

# Define upper and lower bounds for the yellow color in HSV
lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([30, 255, 255])

# Create binary masks for each color
mask_blue = cv.inRange(hsv_image, lower_blue, upper_blue)
mask_red1 = cv.inRange(hsv_image, lower_red1, upper_red1)
mask_red2 = cv.inRange(hsv_image, lower_red2, upper_red2)
mask_yellow = cv.inRange(hsv_image, lower_yellow, upper_yellow)

# Combine the masks for different colors
combined_mask = mask_blue | mask_red1 | mask_red2 | mask_yellow

# Morphological operations to clean up the mask
kernel = np.ones((5, 5), np.uint8)
combined_mask = cv.morphologyEx(combined_mask, cv.MORPH_OPEN, kernel)
combined_mask = cv.morphologyEx(combined_mask, cv.MORPH_CLOSE, kernel)

# Find contours in the binary mask
contours, _ = cv.findContours(combined_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# Filter contours based on area to find the largest contour (traffic sign)
largest_contour = max(contours, key=cv.contourArea)

# Create a mask for the traffic sign region
sign_mask = np.zeros_like(img)
cv.drawContours(sign_mask, [largest_contour], -1, (255, 255, 255), thickness=cv.FILLED)

# Extract the segmented traffic sign from the original image using the mask
seg_traffic_sign = cv.bitwise_and(img, sign_mask)

# Perform edge detection on the segmented traffic sign
edges = cv.Canny(seg_traffic_sign, 100, 200)  # Adjust the thresholds as needed

# Find contours in the edge-detected segmented traffic sign
contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# Filter contours to find the largest contour in the edge-detected segmented traffic sign
largest_contour_seg = max(contours, key=cv.contourArea)

# Draw the largest contour on a black image
contour_image = np.zeros_like(seg_traffic_sign)
cv.drawContours(contour_image, [largest_contour_seg], -1, (255, 0, 0), thickness=cv.FILLED)

# Merge the largest contour with the segmented traffic sign
result = cv.addWeighted(seg_traffic_sign, 1, contour_image, 1, 0)

# Display the result
cv.imshow('Merged Image', result)
cv.waitKey(0)
cv.destroyAllWindows()


## Draw bounding box

In [6]:
# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate through the contours and find bounding boxes
for contour in contours:
    x, y, w, h = cv.boundingRect(contour)
    bounding_boxes.append((x, y, x + w, y + h))  # Format: (x1, y1, x2, y2)

# Draw bounding boxes on the original image
img_with_boxes = result.copy()
for box in bounding_boxes:
    x1, y1, x2, y2 = box
    cv.rectangle(img_with_boxes, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Display or save the image with bounding boxes
cv.imshow("Image with Bounding Boxes", img_with_boxes)
cv.waitKey(0)
cv.destroyAllWindows()

## IoU and Ground truth

In [7]:
import cv2 as cv

# Calculate the IoU between two bounding boxes
def computeIOU(boxA, boxB):
    x_start = max(boxA[0], boxB[0])
    y_start = max(boxA[1], boxB[1])
    x_end = min(boxA[2], boxB[2])
    y_end = min(boxA[3], boxB[3])
    
    interArea = max(0, x_end - x_start + 1) * max(0, y_end - y_start + 1)
    areaA = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    areaB = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    
    return interArea / (areaA + areaB - interArea)

# Extract the bounding box of the segmented region
x, y, w, h = cv.boundingRect(largest_contour)
seg_region_box = (x, y, x + w, y + h)

# Define the ground truth bounding box
# Replace with the actual ground truth bounding box coordinates
ground_truth_box = (55, 29, 172, 163)  # Format: (x1, y1, x2, y2)

# Create a copy of the image with bounding boxes
img_ground_truth = img_with_boxes.copy()

# Draw the ground truth bounding box on the image
x1, y1, x2, y2 = ground_truth_box
cv.rectangle(img_ground_truth, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red color

# Calculate the IoU between the segmented region box and ground truth box
iou = computeIOU(seg_region_box, ground_truth_box)

# Print the IoU value
print("IoU:", iou)

# Display the image with the ground truth bounding box
cv.imshow("Image with Ground Truth", img_ground_truth)
cv.waitKey(0)
cv.destroyAllWindows()


IoU: 0.9842961301177791


In [ ]:
#dont use the bolow code

## circle

In [26]:
# Replace 'image_path' with the path to your input image
image = cv.imread('images/000_0002.png')


In [27]:
# Convert the image to the HSV color space
hsv_image = cv.cvtColor(image, cv.COLOR_BGR2HSV)

# Define the lower and upper bounds for the red color in HSV
lower_red = np.array([0, 100, 100])
upper_red = np.array([10, 255, 255])

# Create a binary mask where the red color is white and everything else is black
mask = cv.inRange(hsv_image, lower_red, upper_red)

# Apply morphological operations to clean up the mask (optional)
kernel = np.ones((5, 5), np.uint8)
mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)

# Find contours in the binary mask
contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# Filter contours based on area to find the circular traffic sign
circular_sign = None
for contour in contours:
    perimeter = cv.arcLength(contour, True)
    approx = cv.approxPolyDP(contour, 0.04 * perimeter, True)
    if len(approx) == 8:  # Assuming a circular sign with 8 corners
        circular_sign = contour
        break

# Create a mask for the circular traffic sign region
sign_mask = np.zeros_like(image)
cv.drawContours(sign_mask, [circular_sign], -1, (255, 255, 255), thickness=cv.FILLED)

# Extract the segmented circular sign from the original image using the mask
segmented_circular_sign = cv.bitwise_and(image, sign_mask)


In [28]:
# Display only the segmented circular sign without any background
cv.imshow('Segmented Circular Traffic Sign', segmented_circular_sign)
cv.waitKey(0)
cv.destroyAllWindows()
